# 📹 MediaTools Autopilot (TikTok + Instagram Viral Pipeline)
Automated clip factory for safe viral content.

Steps:
0. Install dependencies
1. Mount Google Drive
2. Configure settings + sources
3. Prepare folders
4. Download videos
5. Cut clips (22–32s)
6. Subtitles (viral style)
7. Watermark (logo + handle)
8. Captions + hashtags
9. Save/Export

In [ ]:
# STEP 0 — Install dependencies
!apt-get -y update >/dev/null
!apt-get -y install ffmpeg >/dev/null
!pip -q install yt-dlp ffmpeg-python openai-whisper srt numpy pandas >/dev/null
print('✅ Dependencies installed')

In [ ]:
# STEP 1 — Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('✅ Drive mounted')

In [ ]:
# STEP 2 — Settings + Safe Sources
HANDLE='@thedronemenace'
OUTPUT_DIR='Autopost_Clips'
CLIPS_PER_RUN=10
MIN_CLIP_SEC=22
MAX_CLIP_SEC=32
USE_LOGO=True
LOGO_PATH='/content/drive/MyDrive/Autopost_Clips/assets/logo.png'
LOGO_POS='bottom-right'
LOGO_WIDTH=220
SOURCES=[
 'https://www.youtube.com/@PowerfulJRE/videos',
 'https://www.youtube.com/@lexfridman/videos',
 'https://www.youtube.com/@theDiaryOfACEO/videos',
 'https://www.youtube.com/@stanfordgsb/videos',
 'https://www.youtube.com/@TED/videos',
 'https://www.youtube.com/@UniversityofOxford/videos']
print('✅ Settings loaded')

In [ ]:
# STEP 3 — Prepare folders
import os
WORK='/content/work'; RAW=os.path.join(WORK,'raw'); CUT=os.path.join(WORK,'cut'); SUBS=os.path.join(WORK,'subs'); OUT='/content/drive/MyDrive/'+OUTPUT_DIR
for d in (WORK,RAW,CUT,SUBS,OUT): os.makedirs(d,exist_ok=True)
print('✅ Folders ready')

In [ ]:
# STEP 4 — Download recent videos
import subprocess
from glob import glob
MAX_VIDS_PER_SOURCE=3
def download_from_source(url):
    cmd=['yt-dlp','--no-warnings','--ignore-errors','--dateafter','now-1month','-f','bv*[ext=mp4]+ba[ext=m4a]/b[ext=mp4]/b','-o',os.path.join(RAW,'%(uploader)s__%(title).80B__%(id)s.%(ext)s'),'--max-downloads',str(MAX_VIDS_PER_SOURCE),url]
    subprocess.run(cmd,check=False)
for src in SOURCES: download_from_source(src)
raw_files=sorted(glob(os.path.join(RAW,'*.mp4')))
print('✅ Raw videos:',len(raw_files))

In [ ]:
# STEP 5 — Cut clips (22–32s)
import json,random
def get_duration(path):
    cmd=['ffprobe','-v','error','-print_format','json','-show_format','-show_streams',path]
    out=subprocess.check_output(cmd).decode('utf-8','ignore')
    return float(json.loads(out)['format']['duration'])
def pick_segment(duration,min_sec,max_sec):
    length=random.randint(min_sec,max_sec)
    if duration<=length+2: return 0,max(5,min(duration-1,length))
    start=random.uniform(0,max(0,duration-length-1)); return start,length
made=0; random.shuffle(raw_files)
for src in raw_files:
  if made>=CLIPS_PER_RUN: break
  dur=get_duration(src); ss,ll=pick_segment(dur,MIN_CLIP_SEC,MAX_CLIP_SEC)
  base=os.path.splitext(os.path.basename(src))[0]; cut_path=os.path.join(CUT,base+'__clip.mp4')
  subprocess.run(['ffmpeg','-ss',str(ss),'-t',str(ll),'-i',src,'-c:v','libx264','-preset','veryfast','-crf','23','-c:a','aac','-y',cut_path],check=False)
  if os.path.exists(cut_path): print('✔ Cut',cut_path); made+=1
print('✅ Total cuts',made)

In [ ]:
# STEP 6 — Subtitles (viral style)
import whisper,srt,datetime as dt
model=whisper.load_model('small')
cut_files=sorted(glob(os.path.join(CUT,'*.mp4')))
for cut in cut_files:
  base=os.path.splitext(os.path.basename(cut))[0]; srt_path=os.path.join(SUBS,base+'.srt')
  result=model.transcribe(cut,fp16=False,language='en')
  subs=[]
  for i,seg in enumerate(result.get('segments',[])):
    start=dt.timedelta(seconds=max(seg.get('start',0),0)); end=dt.timedelta(seconds=max(seg.get('end',0),0)); text=str(seg.get('text','')).strip()
    subs.append(srt.Subtitle(index=i+1,start=start,end=end,content=text))
  with open(srt_path,'w') as f: f.write(srt.compose(subs))
  print('✔ Subtitles saved',srt_path)

In [ ]:
# STEP 7 — Watermark (logo + handle)
for cut in cut_files:
  base=os.path.splitext(os.path.basename(cut))[0]; srt_path=os.path.join(SUBS,base+'.srt'); out_path=os.path.join(OUT,base+'_final.mp4')
  filter_chain=f"subtitles='{srt_path}':force_style='FontName=DejaVuSans-Bold,Fontsize=28,Outline=2,Shadow=1'"
  filter_chain+=f",drawtext=text='{HANDLE}':x=20:y=20:fontsize=24:fontcolor=white:borderw=2"
  if USE_LOGO and os.path.exists(LOGO_PATH): filter_chain+=f",movie={LOGO_PATH}[logo];[in][logo]overlay=W-w-20:H-h-20"
  subprocess.run(['ffmpeg','-i',cut,'-vf',filter_chain,'-c:a','copy','-y',out_path],check=False)
  print('✔ Watermarked',out_path)

In [ ]:
# STEP 8 — Captions + hashtags
HOOKS=['Would you dare this? 👀','This blew my mind 🤯','Wait till the end 👇','They didn’t want you to see this 👀🔥','The truth nobody is showing you 😳','Caught on camera… 👁️','Don’t blink or you’ll miss it 👇']
HASHTAGS_TT=[['#fyp','#viral','#trending','#xyzbca','#foryou'],['#mustwatch','#clips','#tiktokviral','#funnyvideo','#reels'],['#storytime','#motivation','#wow','#viralvideo','#crazy']]
HASHTAGS_IG=[['#fyp #viral #trending #explorepage #instagramreels #foryou #mustwatch #viralvideo #funnyreels #sharethis #crazyclip #entertainment #wow #epic #insane #clipoftheday #dronevideo #dailyreels #watchnow #socialbuzz'],['#reelsinstagram #instagood #tiktokvideo #viralpost #discover #sharethisnow #funnyclips #epicmoments #viralmoment #mustsee #wild #crazyvideo #clipviral #reelsoftheday #shockfactor #entertainmenthub #trendingnow #insanemoments #exploremore #viralhub'],['#explorepage #viralclips #instagramviral #trendalert #mustwatchnow #viraltoday #crazyclipsoftheday #contentviral #socialtrend #omg #unreal #videoviral #shockvideo #dronefootage #clipcentral #epicreel #viralexplosion #watchthis #topclip #insanereel']]
finals=sorted(glob(os.path.join(OUT,'*_final.mp4')))
for vid in finals:
  base=os.path.splitext(os.path.basename(vid))[0]; hook=random.choice(HOOKS); tt=random.choice(HASHTAGS_TT); ig=random.choice(HASHTAGS_IG)
  caption=f"{hook}\n\n{HANDLE}\n\nTikTok: {' '.join(tt)}\n\nInstagram: {ig[0]}"
  with open(os.path.join(OUT,base+'.txt'),'w') as f: f.write(caption)
  print('✔ Caption file',base+'.txt')
print('✅ All captions generated')

In [ ]:
# STEP 9 — Done
print('🎉 Run complete! Files saved to',OUT)